In [72]:
import numpy as np 
import sympy as sp
from typing import Tuple
from typing import List

In [112]:
class Flat_layout: 
  def __init__ (self, shape: Tuple[int], stride: Tuple[int]): 
    self.rank = len(shape) 
    assert len(shape) == len(stride)
    self.size = np.prod(shape).item() 
    self.co_size = (np.sum((np.array(shape)-1)*np.array(stride)) + 1).item()
    self.shape = shape 
    self.stride = stride 
    self.colex_stride = tuple([np.prod(np.array(shape)[0:i]).item() for i in range(0,self.rank)]) #np already defines empty product as 1. 
    self.colex_map = lambda t: sum(tuple(self.colex_stride[i]*t[i] for i in range(self.rank)))
    self.colex_inv_map = lambda x: tuple((x//self.colex_stride[i])%self.shape[i] for i in range(self.rank))
    self.co_ordinate_map = lambda t: sum(tuple(self.stride[i]*t[i] for i in range(self.rank)))
    self.layout_map = lambda x: self.co_ordinate_map(self.colex_inv_map(x))
    
    self.domain = np.arange(self.size) 
    self.co_ordinate_tensor = np.moveaxis(np.indices(self.shape), 0, -1)
    self.range = np.arange(self.co_size)
    
  def realize(self): 
    self.co_ordinate_map_tensor = np.dot(self.co_ordinate_tensor, np.array(self.stride))
    ss_array = np.array(self.colex_stride).reshape(self.rank,1)
    s_array = np.array(self.shape).reshape(self.rank,1) 
    d_array = np.array(self.stride).reshape(self.rank,1)
    self.layout_map_array = np.sum(((self.domain.reshape(1,self.size)//ss_array) % s_array)*d_array, axis = 0)
    
      

In [113]:
F = Flat_layout((3,4),(4,1))

In [114]:
F.realize()

In [115]:
F.layout_map_array

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

In [116]:
F.co_ordinate_map_tensor

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])